In [1]:
!pip install transformers torch scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# Cấu hình thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

2025-06-12 13:21:51.575231: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749734511.776438      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749734511.835695      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


device(type='cuda')

In [3]:
df = pd.read_csv("/kaggle/input/tiktok-comments/tiktok_comments_balanced.csv")
df.head()

,line_number,text,emotion_label
0,2679,sợ thật,3
1,26129,ối dồi ôi fpt shop cháy tới đó mệt_á,2
2,20113,xem mà khóc thương_k chịu dc,2
3,17380,đàn_bà sống thọ hơn đàn_ông là vậy k,0
4,18200,cô ấy già đi nhiều quá vẫn nhớ ảnh chụp cô lúc...,2


In [4]:
label_mapping = {
    0: "Vui vẻ",        # tích cực
    1: "Tức giận",      # negative (giận dữ, bực tức)
    2: "Buồn",          # sadness
    3: "Sợ hãi",        # lo lắng, hoảng loạn
    4: "Trung lập"      # neutral
}
df["label_text"] = df["emotion_label"].map(label_mapping)

le = LabelEncoder()
df['label'] = le.fit_transform(df['label_text'])
num_classes = len(le.classes_)


df

,line_number,text,emotion_label,label_text,label
0,2679,sợ thật,3,Sợ hãi,1
1,26129,ối dồi ôi fpt shop cháy tới đó mệt_á,2,Buồn,0
2,20113,xem mà khóc thương_k chịu dc,2,Buồn,0
3,17380,đàn_bà sống thọ hơn đàn_ông là vậy k,0,Vui vẻ,4
4,18200,cô ấy già đi nhiều quá vẫn nhớ ảnh chụp cô lúc...,2,Buồn,0
...,...,...,...,...,...
14448,26549,nhìn sợ thế,3,Sợ hãi,1
14449,19664,khổ thân hai nhà bên cạnh,2,Buồn,0
14450,27967,hãi lun vk ơi,3,Sợ hãi,1
14451,7682,vũ,4,Trung lập,2


In [5]:
# Chia dữ liệu
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
MAX_LEN = 64

def tokenize(texts):
    input_ids, attention_masks = [], []
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=MAX_LEN,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Token hóa
train_ids, train_masks = tokenize(train_texts.tolist())
val_ids, val_masks = tokenize(val_texts.tolist())

train_labels = torch.tensor(train_labels.tolist())
val_labels = torch.tensor(val_labels.tolist())

# DataLoader
train_data = TensorDataset(train_ids, train_masks, train_labels)
val_data = TensorDataset(val_ids, val_masks, val_labels)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [6]:
# Load BERT và LSTM

class BertLSTMClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BertLSTMClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.lstm = nn.LSTM(input_size=768, hidden_size=128, num_layers=1,
                            batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(128 * 2, num_classes)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        lstm_out, _ = self.lstm(bert_output.last_hidden_state)
        final_output = lstm_out[:, -1, :]
        dropped = self.dropout(final_output)
        logits = self.fc(dropped)
        return logits




In [9]:
# Huấn luyện
model = BertLSTMClassifier(num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"📈 Epoch {epoch+1}/{EPOCHS} - Loss: {avg_loss:.4f}")

Epoch 1: 100%|██████████| 362/362 [00:52<00:00,  6.87it/s]


📈 Epoch 1/10 - Loss: 1.4124


Epoch 2: 100%|██████████| 362/362 [00:48<00:00,  7.42it/s]


📈 Epoch 2/10 - Loss: 1.2836


Epoch 3: 100%|██████████| 362/362 [00:50<00:00,  7.17it/s]


...

📈 Epoch 8/10 - Loss: 1.1354


Epoch 9: 100%|██████████| 362/362 [00:49<00:00,  7.28it/s]


📈 Epoch 9/10 - Loss: 1.1322


Epoch 10: 100%|██████████| 362/362 [00:50<00:00,  7.24it/s]

📈 Epoch 10/10 - Loss: 1.1021


**Đánh giá**

In [11]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

model.eval()  # dùng model đã định nghĩa (BertLSTMClassifier)

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Đánh giá"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Đánh giá
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Accuracy: {acc:.4f}\n")

print("📋 Classification Report:")
print(classification_report(
    all_labels, 
    all_preds, 
    target_names=le.classes_.astype(str), 
    digits=4
))

print("📊 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

Đánh giá: 100%|██████████| 91/91 [00:11<00:00,  7.94it/s]


✅ Accuracy: 0.5434

📋 Classification Report:
              precision    recall  f1-score   support

        Buồn     0.5318    0.5479    0.5397       626
      Sợ hãi     0.7210    0.6661    0.6925       551
   Trung lập     0.5420    0.5390    0.5405       551
    Tức giận     0.5239    0.5205    0.5222       611
      Vui vẻ     0.4227    0.4457    0.4339       552

    accuracy                         0.5434      2891
   macro avg     0.5483    0.5438    0.5457      2891
weighted avg     0.5473    0.5434    0.5451      2891

📊 Confusion Matrix:
[[343  28  70  92  93]
 [ 54 367  33  67  30]
 [ 48  31 297  56 119]
 [100  54  45 318  94]
 [100  29 103  74 246]]


Test model

In [14]:
def predict_sentiment(texts):
    if isinstance(texts, str):
        texts = [texts]

    # Token hóa
    encoded = tokenizer.batch_encode_plus(
        texts,
        add_special_tokens=True,
        max_length=64,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    # Dự đoán
    model.eval()
    with torch.no_grad():
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(logits, dim=1)

    return le.inverse_transform(preds.cpu().numpy())

In [16]:
test_inputs = [
    "Tự nhiên ra đường cái k đụng chạm j ai củng sợ ngang😑",
    "Lúc đầu thì ""Tùng Bò đâm, xong 1 lúc sau thì ""vào tù nha""😂",
    "bắt quá nhanh 🥰 Công An Việt Nam mình quá giỏi, xuất sắc 🥰"
]

predicted_labels = predict_sentiment(test_inputs)

for text, label in zip(test_inputs, predicted_labels):
    print(f"📝 \"{text}\" → 💬 Dự đoán: {label}")


📝 "Tự nhiên ra đường cái k đụng chạm j ai củng sợ ngang😑" → 💬 Dự đoán: Sợ hãi
📝 "Lúc đầu thì Tùng Bò đâm, xong 1 lúc sau thì vào tù nha😂" → 💬 Dự đoán: Tức giận
📝 "bắt quá nhanh 🥰 Công An Việt Nam mình quá giỏi, xuất sắc 🥰" → 💬 Dự đoán: Vui vẻ


In [ ]:
import zipfile
torch.save(model, "bert_lstm_full_model.pt")


with zipfile.ZipFile("bert_lstm_model.zip", "w") as zipf:
    zipf.write("bert_lstm_full_model.pt")


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

model = torch.load("/kaggle/working/bert_lstm_full_model.pt", map_location=device, weights_only=False)
model.eval()

text = "Tự nhiên ra đường cái k đụng chạm j ai củng sợ ngang😑"

encoded = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=64,
    truncation=True,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt'
)

input_ids = encoded['input_ids'].to(device)
attention_mask = encoded['attention_mask'].to(device)


with torch.no_grad():
    logits = model(input_ids, attention_mask)
    pred = torch.argmax(logits, dim=1).item()

print("✅ Mã lớp dự đoán:", pred)
